In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Step 1: Load your data
store = pd.read_csv('store.csv')
train = pd.read_csv('train.csv', low_memory=False)

# Step 2: Merge the DataFrames on the 'Store' column
trainStore = train.merge(store, on='Store').dropna()

# Step 3: Encode categorical variables
label_encoder = LabelEncoder()
for column in trainStore.columns:
    if trainStore[column].dtype == 'object':
        trainStore[column] = label_encoder.fit_transform(trainStore[column])

# Step 4: Replicate the dataset to make it larger
replicated_data = pd.concat([trainStore] * 10, ignore_index=True)

# Step 5: Split the data into input features (X) and target variable (y)
X = replicated_data.drop(['Sales', 'Date'], axis=1)
y = replicated_data['Sales']

# Step 6: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 8: Move data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device)

# Step 9: Convert data to PyTorch tensors and create DataLoader
class SalesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define batch size
batch_size = 8192

train_dataset = SalesDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Step 10: Define your model with more complexity
class SalesModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.l1 = nn.Linear(input_size, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 1)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()

    def forward(self, xb):
        xb = self.relu(self.batch_norm1(self.l1(xb)))
        xb = self.relu(self.batch_norm2(self.l2(xb)))
        xb = self.l3(xb)
        return xb

# Step 11: Define the optimizer and loss function
model = SalesModel(input_size=X_train_tensor.shape[1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_func = nn.MSELoss()

# Step 12: Implement custom PyTorch Estimator
class PyTorchEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, model, optimizer, loss_func, n_epochs=20):
        self.model = model
        self.optimizer = optimizer
        self.loss_func = loss_func
        self.n_epochs = n_epochs

    def fit(self, X, y):
        self.model.train()
        for epoch in range(self.n_epochs):
            losses_in_epoch = []
            for x_batch, y_batch in train_loader:

                y_hat = self.model(x_batch.to(device))
                loss = self.loss_func(y_hat, y_batch.to(device))
                losses_in_epoch.append(loss.item())



                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            epoch_loss = sum(losses_in_epoch) / len(losses_in_epoch)
            print(f"Epoch {epoch+1}/{self.n_epochs} | Loss: {epoch_loss:.4f}")

        return self

    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
            preds = self.model(X_tensor).cpu().numpy()
        return preds

# Step 13: Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_epochs': [20, 30, 40]
}

estimator = PyTorchEstimator(model, optimizer, loss_func)

grid_search = GridSearchCV(estimator, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Step 14: Get best parameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)


Epoch 1/20 | Loss: 41374941.6151
Epoch 2/20 | Loss: 41375047.2429
Epoch 3/20 | Loss: 41374922.0063
Epoch 4/20 | Loss: 41375029.6151
Epoch 5/20 | Loss: 41374741.2492
Epoch 6/20 | Loss: 41376167.4574
